# Finetuned GPT-4o-mini

## Preparation

In [1]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [2]:
import time
import json
import os
from openai import OpenAI
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score, recall_score

In [3]:
if not os.path.exists("data"):
  !pip install github-clone
  !ghclone https://github.com/yiw008/nondet-project/tree/main/data

  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=459559035a5643e2b7de5f8265fa34857264ffc898cc7a2526729a2e7b8e611a
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
Cloning into 'data'...
done.


In [4]:
def str_to_bool(string):
  if string == "True":
    return True
  elif string == "False":
    return False
  return False

In [5]:
api_key = "" # TODO
os.environ['OPENAI_API_KEY'] = api_key

In [6]:
client = OpenAI()
our_model = "gpt-4o-mini-2024-07-18"

In [7]:
def test_project(project_name):
  print(f"Test project: {project_name}")
  test_set = []
  y_test = []
  with open(f"data/{project_name}/test_set.jsonl", "r") as file:
    for line in file:
      data = json.loads(line)
      test_set.append(data['messages'])
      y_test.append(str_to_bool(data['messages'][2]['content']))

  training_response = client.files.create(
    file=open(f"data/{project_name}/training_set.jsonl", "rb"),
    purpose="fine-tune"
  )
  training_file_id = training_response.id

  validation_response = client.files.create(
    file=open(f"data/{project_name}/validation_set.jsonl", "rb"),
    purpose="fine-tune"
  )
  validation_file_id = validation_response.id

  start = time.time()

  fine_tuning_job_response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=our_model
  )

  fine_tuning_job_id = fine_tuning_job_response.id
  print(f"Fine tuning job ID: {fine_tuning_job_id}")

  status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)
  while status.status not in ["succeeded", "failed"]:
    time.sleep(1)
    status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)

  end = time.time()
  print(f"Finetuning time (Not accurate): {end - start:.4f} seconds")

  status = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)
  print(f"Status: {status}")
  print(f"Created at: {status.created_at}")
  print(f"Finished at: {status.finished_at}")
  print(f"Hyperparams: {status.hyperparameters}")

  fine_tuned_model = status.fine_tuned_model

  y_pred = []
  start = time.time()

  for i in range(len(test_set)):
    completion = client.chat.completions.create(
      model=fine_tuned_model,
      messages=test_set[i]
    )
    y_pred.append(str_to_bool(completion.choices[0].message.content))

  end = time.time()
  print(f"Testing time: {end - start:.4f} seconds")

  print("Y_test:")
  print(y_test)
  print("Y_pred:")
  print(y_pred)

  confusion_matrix_res = confusion_matrix(y_test, y_pred, labels=[False, True])
  print("Confusion Matrix:")
  print(confusion_matrix_res)

  accuracy = accuracy_score(y_test, y_pred)
  print(f"Accuracy: {accuracy}")

  precision = precision_score(y_test, y_pred)
  print(f"Precision: {precision}")

  f1 = f1_score(y_test, y_pred)
  print(f"F1 Score: {f1}")

  recall = recall_score(y_test, y_pred)
  print(f"Recall: {recall}")

## Let's Go

In [8]:
test_project("Butter.MAS.PythonAPI")

Test project: Butter.MAS.PythonAPI
Fine tuning job ID: ftjob-7yHUjnEilc8IPH4E0tIgCQyB
Finetuning time (Not accurate): 2804.7684 seconds
Status: FineTuningJob(id='ftjob-7yHUjnEilc8IPH4E0tIgCQyB', created_at=1732168457, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AVv9xOpN', finished_at=1732171251, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=['file-vaBmMlmDPam16AGEsONhdkRL'], seed=601102525, status='succeeded', trained_tokens=294717, training_file='file-JnNiarLmzoW0tuwY7GpIJcEC', validation_file='file-s2SuhUhH2vonFTLQ4FIRBLTm', estimated_finish=None, integrations=[], user_provided_suffix=None)
Created at: 1732168457
Finished at: 1732171251
Hyperparams: Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8)
Testing time: 93.3191 seconds
Y_test:
[True, T

In [9]:
test_project("centreon-sdk-python")

Test project: centreon-sdk-python
Fine tuning job ID: ftjob-W5jvFbS0W1rijcFw8T29BpR4
Finetuning time (Not accurate): 2528.4124 seconds
Status: FineTuningJob(id='ftjob-W5jvFbS0W1rijcFw8T29BpR4', created_at=1732171363, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AVvqOUMR', finished_at=1732173883, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=['file-j0Z8jK9IuthgVBQscpF0wqT1'], seed=201443963, status='succeeded', trained_tokens=289179, training_file='file-OprmnZoIaVAhfSnJnGYm9K2p', validation_file='file-vXYaY0gc1iGfLxhgHORtmMOR', estimated_finish=None, integrations=[], user_provided_suffix=None)
Created at: 1732171363
Finished at: 1732173883
Hyperparams: Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8)
Testing time: 77.1901 seconds
Y_test:
[True, Tr

In [10]:
test_project("cloudnetpy")

Test project: cloudnetpy
Fine tuning job ID: ftjob-00Ps7cdUyyQY8hfZChXJ1QMJ
Finetuning time (Not accurate): 2442.3035 seconds
Status: FineTuningJob(id='ftjob-00Ps7cdUyyQY8hfZChXJ1QMJ', created_at=1732173970, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AVwV2kcT', finished_at=1732176402, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=['file-7DFHL46jiI00zlZIKBSu9gyf'], seed=1690680030, status='succeeded', trained_tokens=284091, training_file='file-aMO0mFiRSe8PAKx86lJoxqfZ', validation_file='file-X1y58cSQ6PmFDJMUTPRKXAHC', estimated_finish=None, integrations=[], user_provided_suffix=None)
Created at: 1732173970
Finished at: 1732176402
Hyperparams: Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8)
Testing time: 74.3200 seconds
Y_test:
[True, True, True

In [11]:
test_project("crom")

Test project: crom
Fine tuning job ID: ftjob-75TCWATlBiDek1kUs7Y5hGhz
Finetuning time (Not accurate): 2555.8389 seconds
Status: FineTuningJob(id='ftjob-75TCWATlBiDek1kUs7Y5hGhz', created_at=1732176488, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AVxBUEos', finished_at=1732179035, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=['file-RCRgE4Z3S4zWwxbwoVxdYRup'], seed=1135616795, status='succeeded', trained_tokens=283569, training_file='file-kykOzvvwttp47Hn5AcRPomGg', validation_file='file-Gu69RFZZ6aJrZebefcG8UPA5', estimated_finish=None, integrations=[], user_provided_suffix=None)
Created at: 1732176488
Finished at: 1732179035
Hyperparams: Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8)
Testing time: 83.2016 seconds
Y_test:
[True, True, True, True

In [12]:
test_project("easypy")

Test project: easypy
Fine tuning job ID: ftjob-d7xVHZ9PFvow8pQ85tiJ7yug
Finetuning time (Not accurate): 2552.7743 seconds
Status: FineTuningJob(id='ftjob-d7xVHZ9PFvow8pQ85tiJ7yug', created_at=1732179129, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AVxs1lZn', finished_at=1732181671, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=['file-dqqSa1EdqwXthNW1K3rS5d6h'], seed=1903473815, status='succeeded', trained_tokens=295728, training_file='file-El53NfJim5olnWpgEb6ay6D6', validation_file='file-V8ibALyUaTQhkl4Ao3Y8tyfU', estimated_finish=None, integrations=[], user_provided_suffix=None)
Created at: 1732179129
Finished at: 1732181671
Hyperparams: Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8)
Testing time: 76.2123 seconds
Y_test:
[True, True, True, Tr

In [13]:
test_project("eppy")

Test project: eppy
Fine tuning job ID: ftjob-jKUWMTmPgaCHp9gx7ncO3v2c
Finetuning time (Not accurate): 2261.1824 seconds
Status: FineTuningJob(id='ftjob-jKUWMTmPgaCHp9gx7ncO3v2c', created_at=1732181760, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AVyTl1hu', finished_at=1732184011, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=['file-dcpFtN7YQa3qgsQd7KVZZ0lE'], seed=1816702908, status='succeeded', trained_tokens=229578, training_file='file-SoZDe9KcVudGTYUFXTONkg2V', validation_file='file-fTiE0GHe75nPaI2ExJJoh5I4', estimated_finish=None, integrations=[], user_provided_suffix=None)
Created at: 1732181760
Finished at: 1732184011
Hyperparams: Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8)
Testing time: 105.0615 seconds
Y_test:
[True, True, True, Tru

In [14]:
test_project("reframe")

Test project: reframe
Fine tuning job ID: ftjob-cqg49ZGiMy1wZLaDlKGqiBVp
Finetuning time (Not accurate): 2601.3983 seconds
Status: FineTuningJob(id='ftjob-cqg49ZGiMy1wZLaDlKGqiBVp', created_at=1732184127, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AVzBQunL', finished_at=1732186719, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=['file-5YFO520cGqmDFTbl88gL7EHq'], seed=1326295836, status='succeeded', trained_tokens=219150, training_file='file-Vw3TC5pGxFnQ67iUIK4xZ2DR', validation_file='file-0h2YU2LA4oEg5ME5ZQsKeBSO', estimated_finish=None, integrations=[], user_provided_suffix=None)
Created at: 1732184127
Finished at: 1732186719
Hyperparams: Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8)
Testing time: 168.6802 seconds
Y_test:
[True, True, True, 

In [15]:
test_project("region_cache")

Test project: region_cache
Fine tuning job ID: ftjob-wV63Mgp9iK8kwRLy9CGNLzZS
Finetuning time (Not accurate): 3056.8858 seconds
Status: FineTuningJob(id='ftjob-wV63Mgp9iK8kwRLy9CGNLzZS', created_at=1732186899, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AW01UyNq', finished_at=1732189946, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=['file-sgPFnxND8uGLqBIRtdftICKM'], seed=103980102, status='succeeded', trained_tokens=322299, training_file='file-goQryNg1aGEnTPxrsHvuXnRr', validation_file='file-oXawGQZRyeBtzAo5Mk68hO1M', estimated_finish=None, integrations=[], user_provided_suffix=None)
Created at: 1732186899
Finished at: 1732189946
Hyperparams: Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8)
Testing time: 93.1479 seconds
Y_test:
[False, False, F

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
test_project("taxi")

Test project: taxi
Fine tuning job ID: ftjob-V2rlWevRrMrQhK5EsOzqENEa
Finetuning time (Not accurate): 3105.0516 seconds
Status: FineTuningJob(id='ftjob-V2rlWevRrMrQhK5EsOzqENEa', created_at=1732190051, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AW0r6xO7', finished_at=1732193145, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=['file-rtQu4AGmIv1Mf6XgtfBGAtjy'], seed=475467753, status='succeeded', trained_tokens=327954, training_file='file-kBGQcPmDGlrgMfECHxhcETpM', validation_file='file-oq6kaPBzrYmB1i5cKGLNiYPM', estimated_finish=None, integrations=[], user_provided_suffix=None)
Created at: 1732190051
Finished at: 1732193145
Hyperparams: Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8)
Testing time: 102.2989 seconds
Y_test:
[False, False, False, F

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
test_project("wkr.py")

Test project: wkr.py
Fine tuning job ID: ftjob-LVH2umE52I1kLIum4EHvXkGD
Finetuning time (Not accurate): 3096.9523 seconds
Status: FineTuningJob(id='ftjob-LVH2umE52I1kLIum4EHvXkGD', created_at=1732193261, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::AW1gi3yt', finished_at=1732196347, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-tI0WpKootnbW2KXQbHxLn75u', result_files=['file-u49sg36LOj6DlSWeVaDtwLMB'], seed=1511335162, status='succeeded', trained_tokens=309702, training_file='file-ocWJCHDoM4OQ7BPL2mkydmeg', validation_file='file-QEMb1hQOZzKh7Jipr3PL7Czn', estimated_finish=None, integrations=[], user_provided_suffix=None)
Created at: 1732193261
Finished at: 1732196347
Hyperparams: Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=1.8)
Testing time: 64.6097 seconds
Y_test:
[False, False, False,

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
